In [1]:
# !pip install -q -U kaggle_environments

# Домашнее задание

ВАМ НЕОБХОДИМО:
* Описать поведение бота, который будет играть с такими же ботами в игру «камень-ножницы-бумага». Поведение бота описывается с помощью функции, которая принимает на вход информацию о прошлых играх. (сигнатура функции приведена в Google Colab, а детальное описание входящих значений доступно по ссылке)
* После описания поведения агентов запустить турнир между ними и проверить, какая стратегия показывает себя лучше всех.

Отметим, что вам необязательно использовать «качественные» стратегии — в этом задании основной упор необходимо сделать на разнообразие (т.е. агенты, которые бы играли, придерживаясь стратегии «только камень» или »только ножницы» – это нормально).

### Импортируем необходимые библиотеки. 
Визуализацию мы пока не используем. Библиотеки для визуализации данных не подгружаем

In [2]:
import numpy as np
import pandas as pd
import random

# import matplotlib.pyplot as plt
# import seaborn as sns

from kaggle_environments import make, evaluate

In [5]:
# pd.__version__

'1.4.4'

In [4]:
# !pip install --upgrade pandas

## Агенты для турнира

### 1 агент, Моностратегия камень - предложенный авторами домашнего задания в качестве примера

Опишем поведение агента, всегда играющего "камень" - это значение 0

In [6]:
%%writefile rock_agent.py

#Example of the simple agent
#0 - rock
#1 - paper
#2 - scissors
def your_agent(observation, configuration):
    return 0

Writing rock_agent.py


### 2 агент, Копирование оппонента - предложенный авторами домашнего задания в качестве примера

Попробуем теперь использовать информацию о прошлых действиях противника. Опишем агента, который производит то же самое действие, что и оппонент на прошлом ходу

In [12]:
%%writefile copy_opponent.py

#Example 
def copy_opponent(observation, configuration):
    #in case we have information about opponent last move
    if observation.step > 0:
        return observation.lastOpponentAction
    #initial step
    else:
        return random.randrange(0, configuration.signs)

Writing copy_opponent.py


### 3 агент. Копия 1 - Моностратегия ножницы

In [7]:
%%writefile scissors.py

def scissors(observation, configuration):
    return 2

Writing scissors.py


### 4 агент. Копия 1 - Моностратегия бумага

In [13]:
%%writefile paper.py

def paper(observation, configuration):
    return 1

Writing paper.py


### 5 агент. Из базового набора агентов - реагирующий на действия оппонента

In [8]:
%%writefile reactionary.py

def reactionary(observation, configuration):
    global last_react_action
    if observation.step == 0:
        last_react_action = random.randrange(0, configuration.signs)
    elif get_score(last_react_action, observation.lastOpponentAction) <= 1:
        last_react_action = (observation.lastOpponentAction + 1) % configuration.signs

    return last_react_action

Writing reactionary.py


### 6 агент. Из базового набора агентов - опирающийся на статистику

In [10]:
%%writefile statistical.py

def statistical(observation, configuration):
    global action_histogram
    if observation.step == 0:
        action_histogram = {}
        return
    action = observation.lastOpponentAction
    if action not in action_histogram:
        action_histogram[action] = 0
    action_histogram[action] += 1
    mode_action = None
    mode_action_count = None
    for k, v in action_histogram.items():
        if mode_action_count is None or v > mode_action_count:
            mode_action = k
            mode_action_count = v
            continue

    return (mode_action + 1) % configuration.signs

Writing statistical.py


### 7 агент. Рандомный агент - судя по Кэглу опирается на равенство Нэша

In [11]:
%%writefile random.py

import random

def random_agent(observation, configuration):
    return random.randint(0, 2)

Writing random.py


### 8 агент. Победитель соревнования на Kaggle - Решающее дерево

In [14]:
%%writefile decision_tree.py

import numpy as np
import collections
from sklearn.tree import DecisionTreeClassifier

def construct_local_features(rollouts):
    features = np.array([[step % k for step in rollouts['steps']] for k in (2, 3, 5)])
    features = np.append(features, rollouts['steps'])
    features = np.append(features, rollouts['actions'])
    features = np.append(features, rollouts['opp-actions'])
    return features

def construct_global_features(rollouts):
    features = []
    for key in ['actions', 'opp-actions']:
        for i in range(3):
            actions_count = np.mean([r == i for r in rollouts[key]])
            features.append(actions_count)
    
    return np.array(features)

def construct_features(short_stat_rollouts, long_stat_rollouts):
    lf = construct_local_features(short_stat_rollouts)
    gf = construct_global_features(long_stat_rollouts)
    features = np.concatenate([lf, gf])
    return features

def predict_opponent_move(train_data, test_sample):
    classifier = DecisionTreeClassifier(random_state=42)
    classifier.fit(train_data['x'], train_data['y'])
    return classifier.predict(test_sample)

def update_rollouts_hist(rollouts_hist, last_move, opp_last_action):
    rollouts_hist['steps'].append(last_move['step'])
    rollouts_hist['actions'].append(last_move['action'])
    rollouts_hist['opp-actions'].append(opp_last_action)
    return rollouts_hist

def warmup_strategy(observation, configuration):
    global rollouts_hist, last_move
    action = int(np.random.randint(3))
    if observation.step == 0:
        last_move = {'step': 0, 'action': action}
        rollouts_hist = {'steps': [], 'actions': [], 'opp-actions': []}
    else:
        rollouts_hist = update_rollouts_hist(rollouts_hist, last_move, observation.lastOpponentAction)
        last_move = {'step': observation.step, 'action': action}
    return int(action)

def init_training_data(rollouts_hist, k):
    for i in range(len(rollouts_hist['steps']) - k + 1):
        short_stat_rollouts = {key: rollouts_hist[key][i:i+k] for key in rollouts_hist}
        long_stat_rollouts = {key: rollouts_hist[key][:i+k] for key in rollouts_hist}
        features = construct_features(short_stat_rollouts, long_stat_rollouts)        
        data['x'].append(features)
    test_sample = data['x'][-1].reshape(1, -1)
    data['x'] = data['x'][:-1]
    data['y'] = rollouts_hist['opp-actions'][k:]
    return data, test_sample

def agent(observation, configuration):
    # hyperparameters
    k = 5
    min_samples = 25
    global rollouts_hist, last_move, data, test_sample
    if observation.step == 0:
        data = {'x': [], 'y': []}
    # if not enough data -> randomize
    if observation.step <= min_samples + k:
        return warmup_strategy(observation, configuration)
    # update statistics
    rollouts_hist = update_rollouts_hist(rollouts_hist, last_move, observation.lastOpponentAction)
    # update training data
    if len(data['x']) == 0:
        data, test_sample = init_training_data(rollouts_hist, k)
    else:        
        short_stat_rollouts = {key: rollouts_hist[key][-k:] for key in rollouts_hist}
        features = construct_features(short_stat_rollouts, rollouts_hist)
        data['x'].append(test_sample[0])
        data['y'] = rollouts_hist['opp-actions'][k:]
        test_sample = features.reshape(1, -1)
        
    # predict opponents move and choose an action
    next_opp_action_pred = predict_opponent_move(data, test_sample)
    action = int((next_opp_action_pred + 1) % 3)
    last_move = {'step': observation.step, 'action': action}
    return action

Writing decision_tree.py


### 9 агент. Простой агент рандомно выбирающий между камнем и бумагой

In [15]:
%%writefile my_random.py

import random

def my_random_agent(observation, configuration):
    return random.randint(0, 1)

Writing my_random.py


### 10 агент. Маркова

In [24]:
%%writefile markov.py

import numpy as np
import collections

def markov_agent(observation, configuration):
    k = 2
    global table, action_seq
    if observation.step % 50 == 0: # refresh table every 50 steps
        action_seq, table = [], collections.defaultdict(lambda: [1, 1, 1])    
    if len(action_seq) <= 2 * k + 1:
        action = int(np.random.randint(3))
        if observation.step > 0:
            action_seq.extend([observation.lastOpponentAction, action])
        else:
            action_seq.append(action)
        return action
    # update table
    key = ''.join([str(a) for a in action_seq[:-1]])
    table[key][observation.lastOpponentAction] += 1
    # update action seq
    action_seq[:-2] = action_seq[2:]
    action_seq[-2] = observation.lastOpponentAction
    # predict opponent next move
    key = ''.join([str(a) for a in action_seq[:-1]])
    if observation.step < 500:
        next_opponent_action_pred = np.argmax(table[key])
    else:
        scores = np.array(table[key])
        next_opponent_action_pred = np.random.choice(3, p=scores/scores.sum()) # add stochasticity for second part of the game
    # make an action
    action = (next_opponent_action_pred + 1) % 3
    # if high probability to lose -> let's surprise our opponent with sudden change of our strategy
    if observation.step > 900:
        action = next_opponent_action_pred
    action_seq[-1] = action
    return int(action)

Writing markov.py


Использованные агенты:
* Моно-стратегия (только камень, только ножницы, только бумага)
* Копирование оппонента
* Реакция на действия оппонента
* Контр-реакция на действия оппонента
* Сбор статистики
* Рандомный - Равенство Нэша
* Маркова
* Победитель соревнования на Кэгле - Решающее дерево
* Простой бот рандомно выбирающий между камнем и бумагой

## Турниры

Воспользуемся функцией evaluate из библиотеки kaggle_environments с помощью которой запустим наших агентов и проведем эксперимент на заданном количестве игр

In [17]:
evaluate(
    "rps", #environment to use - no need to change
    ["rock_agent.py", "copy_opponent"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes 
)

[[0, 0]]

**1)** Первый агент всегда показывает камень, второй копирует действия оппонента. Ничья. Базовый турнир, предложенный авторами ДЗ

In [18]:
evaluate(
    "rps", #environment to use - no need to change
    ["rock_agent.py", "paper.py"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes 
)

[[-99.0, 99.0]]

**2)** Первый агент всегда показывает камень, второй бумагу. Бумага побеждает

In [50]:
evaluate(
    "rps", #environment to use - no need to change
    ["rock_agent.py", "scissors.py"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes 
)

[[99.0, -99.0]]

**3)** Первый агент всегда показывает камень, второй ножницы. Камень побеждает

In [53]:
evaluate(
    "rps", #environment to use - no need to change
    ["scissors.py", "paper.py"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes 
)

[[99.0, -99.0]]

**4)** Первый агент всегда показывает ножницы, второй бумагу. Ножницы побеждают

**Вывод: Моноагенты всегда побеждают или проигрывают в зависимости от оппонента**

In [21]:
evaluate(
    "rps", #environment to use - no need to change
    ["copy_opponent", "reactionary"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes 
)

[[-50.0, 50.0]]

**5)** Первый агент копирует оппонента, второй реагирует на действия оппонента. Реагирующий бот победил.

In [64]:
evaluate(
    "rps", #environment to use - no need to change
    ["copy_opponent", "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes 
)

[[-20.0, 20.0]]

**6)** Первый агент копирует оппонента, второй опирается на статистику. Бот опирающийся на статистику победил.

In [ ]:
evaluate(
     "rps", #environment to use - no need to change
    ["statistical", "reactionary"], #agents to evaluate
     configuration={"episodeSteps": 100} #number of episodes 
)

**7)** Первый агент опирается на статистику, второй реагирует на действия первого. Второй бот победил.

In [24]:
evaluate(
     "rps", #environment to use - no need to change
    ["reactionary", "counter_reactionary"], #agents to evaluate
     configuration={"episodeSteps": 100} #number of episodes 
)

[[0, 0]]

**8)** Первый агент реагирует на второго, второй контр-реагирует на действия первого. Ничья.

In [25]:
evaluate(
     "rps", #environment to use - no need to change
    ["statistical", "counter_reactionary"], #agents to evaluate
     configuration={"episodeSteps": 100} #number of episodes 
)

[[0, 0]]

**9)** Первый агент опирается на статистику, второй контр-реагирует на действия первого. Ничья.

In [26]:
evaluate(
     "rps", #environment to use - no need to change
    ["rock", "counter_reactionary"], #agents to evaluate
     configuration={"episodeSteps": 100} #number of episodes 
)

[[-49.0, 49.0]]

**10)** Первый агент всегда выкидывает камень, второй контр-реагирует на действия первого. Контр-реагирующий бот выйграл.

In [28]:
evaluate(
     "rps", #environment to use - no need to change
    ["paper", "reactionary"], #agents to evaluate
     configuration={"episodeSteps": 100} #number of episodes 
)

[[-97.0, 97.0]]

**11)** Первый агент всегда выкидывает бумагу, второй реагирует на действия первого. Реагирующий бот выйграл.

In [22]:
evaluate(
     "rps", #environment to use - no need to change
    ["random.py", "reactionary"], #agents to evaluate
     configuration={"episodeSteps": 100} #number of episodes 
)

[[0, 0]]

**12)** Первый агент действует в соответствии с равенством Нэша, второй реагирует на действия первого. Ничья!

In [44]:
evaluate(
     "rps", #environment to use - no need to change
    ["markov.py", "reactionary"], #agents to evaluate
     configuration={"episodeSteps": 100} #number of episodes 
)

[[41.0, -41.0]]

**13)** Первый агент действует в соответствии с функцией Маркова, второй реагирует на действия первого. Марков побеждает

In [25]:
evaluate(
     "rps", #environment to use - no need to change
    ["markov.py", "random.py"], #agents to evaluate
     configuration={"episodeSteps": 100} #number of episodes 
)

[[0, 0]]

**14)** Первый агент действует в соответствии с функцией Маркова, второй реагирует в соответствии с равенством Нэша. Ничья!

In [26]:
evaluate(
     "rps", #environment to use - no need to change
    ["markov.py", "rock"], #agents to evaluate
     configuration={"episodeSteps": 100} #number of episodes 
)

[[94.0, -94.0]]

**15)** Первый агент действует в соответствии с функцией Маркова, второй использует моностратегию - камень. Марков порвал камень

In [47]:
evaluate(
     "rps", #environment to use - no need to change
    ["markov.py", "counter_reactionary"], #agents to evaluate
     configuration={"episodeSteps": 100} #number of episodes 
)

[[60.0, -60.0]]

**Вывод: агент Маркова достаточно сильный, но нейтрализуется агентом на базе рандомного агента, который все сводит к ничьей**

## Битва финалистов

In [41]:
evaluate(
     "rps", #environment to use - no need to change
    ["markov.py", "decision_tree.py"], #agents to evaluate
     configuration={"episodeSteps": 600} #number of episodes 
)


[[-116.0, 116.0]]

In [31]:
env = make(
    "rps", 
    configuration={
        "episodeSteps": 100
    }
)
env.run(
    ["markov.py", "decision_tree.py"]
)

env.render(mode="ipython", width=500, height=400)

In [40]:
evaluate(
     "rps", #environment to use - no need to change
    ["random.py", "decision_tree.py"], #agents to evaluate
     configuration={"episodeSteps": 500} #number of episodes 
)

[[0, 0]]

In [35]:
env = make(
    "rps", 
    configuration={
        "episodeSteps": 98
    }
)
env.run(
    ["random.py", "decision_tree.py"]
)

env.render(mode="ipython", width=500, height=400)

In [39]:
evaluate(
     "rps", #environment to use - no need to change
    ["random.py", "markov.py"], #agents to evaluate
     configuration={"episodeSteps": 1000} #number of episodes 
)

[[65.0, -65.0]]

## Вывод: Решающее дерево побеждает чаще, но победа зависит в т.ч. от количества раундов